# SOMBR: Using SOM to Triage Software Bug Reports

## Setup

### Import package dependencies

In [1]:
import os
import re
import sys
import sqlite3
import pandas as pd
import numpy as np
import ipywidgets 
from tqdm.notebook import trange, tqdm
import time
import datetime
import warnings
warnings.filterwarnings("ignore")

import logging


In [2]:
from scipy import stats
import json


In [3]:
import matplotlib.pyplot as plt
import seaborn as sns


%matplotlib inline

sns.set(style="white")
sns.set_context("paper")
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.family'] = ['cmr10', 'sans-serif']
plt.rcParams['axes.formatter.use_mathtext'] = True

centim = 1/2.54  # centimeters in inches
figsize_a5 = (21*centim,14.5*centim)
figsize_a4 = (21*centim,29*centim)


## Account for caption space s cm
def figsize_a5_cap(cs=0): 
    return (21*centim,(14.85-cs)*centim)

def figsize_a4_cap(cs=0): 
    return (21*centim,(29.7-cs)*centim)





In [4]:
if os.path.abspath('..') not in sys.path:
    sys.path.append(os.path.join(os.path.abspath('..')))
    
    print('parent not in path, appending')
    
if os.path.abspath('.') not in sys.path:
    sys.path.append(os.path.join(os.path.abspath('.')))
    
    print('parent not in path, appending')
    
if os.path.abspath('/gcs') not in sys.path:
    sys.path.append(os.path.join(os.path.abspath('/gcs')))
    
    print('gcs not in path, appending')

parent not in path, appending
gcs not in path, appending


In [5]:
from minisom import MiniSom

In [6]:
from IPython.display import Audio

sound_file = './SOUNDS/WAV_Sharp.wav'
final_sound = './SOUNDS/WAV_Enharpment.wav'


In [7]:
def timenow():
    return datetime.datetime.now().strftime("%H:%M:%S")

def play_done(s=sound_file):
    print(timenow())
    display(Audio(s, autoplay=True))
    return

play_done()

13:55:32


In [8]:

logging.basicConfig(filename="SOMLOG-{}.log".format(timenow()), encoding='utf-8', level=logging.DEBUG)

In [9]:
#  for debugging
A_CURRENT_EXP = None
A_UPDATED_EXP = None
A_CURRENT_PREP = None
A_UPDATED_PREP = None

In [10]:
RANDOM_STATE = 0
np.random.seed(RANDOM_STATE)
# np.random.randint(low=1,high=9999,size=4)

random_seeds = [2733, 9846, 3265, 4860]

## Get data from CSVs


In [11]:
CORPORA = ['cassandra',
 'groovy',
 'hadoop',
 'hbase',
 'hibernate',
 'hive',
 'infinispan',
 'jbpm',
 'lucene',
 'spark'
]


PREPR = ['lemma', 'stopwords', 'base', 'lowercase', 'camel', 'camellower']


VECTZR = ['tfidf', 'w2v', 'd2v', 'bert', 'sbert']

METRICS = ['a1', 'ab', 'pr', 'qe', 'te']

metrics_labels = {
    'a1':'Accuracy@1',
    'ab':'Accuracy@BMU',
    'pr':'Precision',
    'te':'Topographic Error',
    'qe':'Quantization Error'
}

results_dest = "results"

src_csv_suffix = "150000"

In [12]:
vectors_base = "./csv/vectors/"

DF_MAP = {}

for p in tqdm(PREPR, desc="PREP", leave=False, position=10):
    DF_MAP[p] = {}
    for ci in tqdm(CORPORA, desc="CORP", leave=False, position=11):
        DF_MAP[p][ci] = {}
        for vi in tqdm(VECTZR, desc="VECT", leave=False, position=12):
            bcv = "{}{}/{}_{}".format(vectors_base, p, ci, vi)
            train_df = pd.read_csv(bcv+"_train.csv")
            test_df = pd.read_csv(bcv+"_test.csv")
            train_y = train_df.iloc[:,-1]
            vi_map = {
                'train_df_path': bcv+"_train.csv",
                'test_df_path': bcv+"_test.csv",
                'train_df': train_df,
                'test_df': test_df,
                'train_y': train_y,
            }
            
            DF_MAP[p][ci][vi] = vi_map


PREP:   0%|          | 0/6 [00:00<?, ?it/s]

CORP:   0%|          | 0/10 [00:00<?, ?it/s]

VECT:   0%|          | 0/5 [00:00<?, ?it/s]

VECT:   0%|          | 0/5 [00:00<?, ?it/s]

VECT:   0%|          | 0/5 [00:00<?, ?it/s]

...

VECT:   0%|          | 0/5 [00:00<?, ?it/s]

VECT:   0%|          | 0/5 [00:00<?, ?it/s]

VECT:   0%|          | 0/5 [00:00<?, ?it/s]

VECT:   0%|          | 0/5 [00:00<?, ?it/s]

VECT:   0%|          | 0/5 [00:00<?, ?it/s]

In [13]:
BASE_SETUP = {
    "label": "Base",
    "vectors_path" : "./csv/vectors/",
    "results_path" : "./results/base/",
    "epochs" : 10000,
    "random_state": 0,
    "learning_rate": 0.7,
    "initialization": 'random',
    "somg_m": 15,
    "somg_n": 15,
}

def new_xsetup(label=None, 
               # vectors_path=None, 
               results_path=None, 
               epochs=None, 
               rs=None, 
               lr=None, 
               initzn=None):
    new_set = BASE_SETUP.copy()
    if label != None:
        new_set['label'] = label
    if results_path != None:
        new_set['results_path'] = results_path
    if epochs != None:
        new_set['epochs'] = epochs
    if rs != None:
        new_set['random_state'] = rs
    if initzn != None:
        new_set['initialization'] = initzn

    return new_set

In [14]:
def get_row_acc(row, wm, targets, k=None):
    """
    Get the predicted user for this row by getting the mode of the users in the row's BMU.
    In case there are multiple modes:
        returns the true label as the prediction if it is in one of the modes;
        otherwise, returns the first mode result.
    """

    ## This contains the documents present in the bmu
    # br_in_bmu = wm[row['bmu']]
    br_in_bmu = None
    try:
        br_in_bmu = wm[row['bmu']]
    except:
        br_in_bmu = []
    
    ## These are the users for each br in bmu
    candidates = []
    if len(br_in_bmu) > 0:
        candidates = targets[br_in_bmu]
        
        if k==1: 
            ## Positive result only if target is in the top-1 mode
            ## This is the list of modes of the users in the bmu
            modes = candidates.mode().values

            ## In case of multiple modes
            if row['target'] in modes:
                return 1
        else:
            ## Positive result if target is in BMU
            if row['target'] in candidates.to_list():
                return 1

    return 0
def get_row_bmu(row, som, mn):
    """
    Returns the longform ID of the BMU for this row
    """
    row_ = row.iloc[:-1]
    return np.ravel_multi_index(som.winner(row_.values),mn)

def get_row_prediction(row, wm, targets):
    """
    Get the predicted user for this row by getting the mode of the users in the row's BMU.
    In case there are multiple modes:
        returns the true label as the prediction if it is in one of the modes;
        otherwise, returns the first mode result.
    """

    ## This contains the documents present in the bmu
    # br_in_bmu = wm[row['bmu']]
    br_in_bmu = None
    try:
        br_in_bmu = wm[row['bmu']]
    except:
        br_in_bmu = []
    
    ## These are the users for each br in bmu
    candidates = []
    if len(br_in_bmu) > 0:
        candidates = targets[br_in_bmu]
    
        ## This is the list of modes of the users in the bmu
        modes = candidates.mode().values
    
        ## In case of multiple modes
        if row['target'] in modes:
            return row['target']
        else:
            return modes[0]
    else:
        ## assigned to empty bmu
        return -1

def get_relevant_count(row, wm, targets):
    """
    Get the predicted user for this row by getting the mode of the users in the row's BMU.
    In case there are multiple modes:
        returns the true label as the prediction if it is in one of the modes;
        otherwise, returns the first mode result.
    """

    ## This contains the indices of the documents present in the bmu
    br_in_bmu = None
    try:
        br_in_bmu = wm[row['bmu']]
    except:
        br_in_bmu = []
        
    
    ## These are the users for each br in bmu
    candidates = []    
    if len(br_in_bmu) > 0:
        candidates = targets[br_in_bmu]
        count = candidates.to_list().count(row['target'])
        return count
    else:
        return 0
        
    
    ## This how many times the target appears in the bmu
    ## which corresponds to the count of relevant documents in IR
    # print(row['target'])
    # print(candidates)
    
    
def get_bmu_size(row, wm):
    """
    Get the predicted user for this row by getting the mode of the users in the row's BMU.
    In case there are multiple modes:
        returns the true label as the prediction if it is in one of the modes;
        otherwise, returns the first mode result.
    """

    ## This contains the number of documents present in the bmu
    try:
        bmu_size = len(wm[row['bmu']])
        return bmu_size
    except:
        return None
    

In [15]:
DF_MAP.keys()

dict_keys(['lemma', 'stopwords', 'base', 'lowercase', 'camel', 'camellower'])

In [16]:
# train_df = 
DF_MAP['lemma'][ci][vi].keys()

dict_keys(['train_df_path', 'test_df_path', 'train_df', 'test_df', 'train_y'])

In [17]:
def get_test_scores(P_MAP, ci, vi, som_params=None):

    train_df = P_MAP[ci][vi]['train_df']
    test_df = P_MAP[ci][vi]['test_df']
    train_y = P_MAP[ci][vi]['train_y']

    p_ = train_df.shape[1]-1
    
    som_ = MiniSom(som_params['somg_m'], 
                   som_params['somg_n'], 
                   p_, 
                   sigma=1.5, 
                   learning_rate=som_params['learning_rate'], 
                   activation_distance='euclidean',
                   topology='hexagonal', 
                   neighborhood_function='gaussian', 
                   random_seed=som_params['random_state']
                 )
    train_vals = train_df.iloc[:,:-1].values
    if som_params['initialization'] == 'pca':
        som_.pca_weights_init(train_vals)
    else:
        som_.random_weights_init(train_vals)
        
    som_.train_batch(train_vals, som_params['epochs'])
    qe = som_.quantization_error(train_vals)
    te = som_.topographic_error(train_vals)
    
    win_map     = som_.win_map(train_vals, return_indices=True)
    winmap_long = { np.ravel_multi_index(i,(som_params['somg_m'],som_params['somg_n'])): win_map[i] for i in win_map}

    results_ = pd.DataFrame(index=test_df.index)
    results_['target']    = test_df.iloc[:,-1]
    results_['bmu']       = test_df.apply(lambda row: get_row_bmu(row, som_, (som_params['somg_m'],som_params['somg_n'])), axis=1)
    results_['pred']      = results_.apply(lambda row: get_row_prediction(row, winmap_long, train_y), axis=1)
    results_['relevant']  = results_.apply(lambda row: get_relevant_count(row, winmap_long, train_y), axis=1)
    results_['bmu_size']  = results_.apply(lambda row: get_bmu_size(row, winmap_long), axis=1)
    results_['precision'] = results_['relevant'] / results_['bmu_size']
    
    results_['acc_1']     = results_.apply(lambda row: get_row_acc(row, winmap_long, train_y, k=1), axis=1)
    results_['acc_bmu']   = results_.apply(lambda row: get_row_acc(row, winmap_long, train_y), axis=1)

    return results_['precision'].mean(), results_['acc_1'].mean(), results_['acc_bmu'].mean(), qe, te
    
    


In [18]:
def run_tests(results_base, vec_path=None, som_params=None ):

    niters = som_params['epochs']
    
    test_results_df_acc1 = pd.DataFrame(index=CORPORA, columns=[])
    test_results_df_accb = pd.DataFrame(index=CORPORA, columns=[])
    test_results_df_prec = pd.DataFrame(index=CORPORA, columns=[])
    test_results_df_qe = pd.DataFrame(index=CORPORA, columns=[])
    test_results_df_te = pd.DataFrame(index=CORPORA, columns=[])
    
    for vz_i in tqdm(VECTZR, desc="Vectzr:", position=2, leave=False):
        test_results_acc1 = {}
        test_results_accb = {}
        test_results_prec = {}
        test_results_qe   = {}
        test_results_te   = {}
        for corp_i in tqdm(CORPORA, position=3, leave=False, desc="Corpus:"):

            p, a1, ab, qe, te = get_test_scores(vec_path, corp_i, vz_i, som_params)

            test_results_acc1[corp_i] = a1
            test_results_accb[corp_i] = ab
            test_results_prec[corp_i] = p
            test_results_qe[corp_i]   = qe
            test_results_te[corp_i]   = te
        test_results_df_acc1[vz_i] = pd.Series(test_results_acc1)
        test_results_df_accb[vz_i] = pd.Series(test_results_accb)
        test_results_df_prec[vz_i] = pd.Series(test_results_prec)
        test_results_df_qe[vz_i]   = pd.Series(test_results_qe)
        test_results_df_te[vz_i]   = pd.Series(test_results_te)

    test_results_df_acc1.to_csv("{}a1_{}.csv".format(results_base, niters))
    test_results_df_accb.to_csv("{}ab_{}.csv".format(results_base, niters))
    test_results_df_prec.to_csv("{}pr_{}.csv".format(results_base, niters))
    test_results_df_qe.to_csv("{}qe_{}.csv".format(results_base, niters))
    test_results_df_te.to_csv("{}te_{}.csv".format(results_base, niters))
    
    # print("{}/te_{}.csv".format(results_base, niters))

    dfs = {
    'acc1': {'df': test_results_df_acc1,
             'label': "Accuracy @ 1"
        },
    'accb': {'df': test_results_df_accb,
         'label': "Accuracy @ BMU"
        },
    'prec': {'df': test_results_df_prec,
         'label': "Precision"
        },
    'qe': {'df': test_results_df_qe,
         'label': "Quantization Error"
        },
    'te': {'df': test_results_df_te,
         'label': "Topographic Error"
        },        
    }
    
    return dfs



In [19]:
EXPERIMENTS = {
    "Base": new_xsetup(),
    "Mini": new_xsetup("Mini", "./mini/", 10),
    "150k" : new_xsetup("SOM 150k", "./SOM_150k/", 150000),
    "50k" : new_xsetup("SOM 50k", "./SOM_50k/", 50000),
    "Random2" : new_xsetup("SOM Random 2", "./SOM_Random2/", rs=random_seeds[0]),
    "Random3" : new_xsetup("SOM Random 3", "./SOM_Random3/", rs=random_seeds[1]),
    "Random4" : new_xsetup("SOM Random 4", "./SOM_Random4/", rs=random_seeds[2]),
    "Random5" : new_xsetup("SOM Random 5", "./SOM_Random5/", rs=random_seeds[3]),
    "PCA" : new_xsetup("SOM PCA", "./SOM_PCA/", initzn="pca"),
    "LR1" : new_xsetup("SOM LR 1", "./SOM_LR_1/", lr=1),
}

EXP_TO_RUN = ['Base', 'Random2', 'Random3', 'Random4', 'Random5', 'PCA', 'LR1']

In [20]:

def run_experiments(e="Base"):    
    prepbar = tqdm(PREPR, desc="Prep: ", position=1, leave=False) 
    experiment_results = {}
    for p in prepbar:
        start_exp = time.time()
        
        prepbar.set_description("Prep: {}".format(p))
        A_CURRENT_PREP = p
        A_UPDATED_PREP = timenow()
        
        results_base = "./results/{}/{}/".format(e,p)
        vectors_path = "./csv/vectors/{}/".format(p)
        if not os.path.exists(results_base):
            os.makedirs(results_base)

        test_results = run_tests(results_base, DF_MAP[p], EXPERIMENTS[e])
        experiment_results[exp] = test_results
        # play_done()
        end_exp = time.time()
        update_log("\t\tPREP {}".format(p), timenow(),  end_exp-start_exp)
    
    return experiment_results
    

# run_experiments(e="Mini")




In [21]:

def update_log(msg, t, d):
    logging.debug("{} at {}; after {} min".format(msg, t, np.round(d/60,4)))
    

In [22]:
all_exps_results = {}

expbar = tqdm(EXP_TO_RUN, desc="Experiment:", leave=False, position=0)
start_out = time.time()
for exp in expbar:
    start_exp = time.time()
    update_log("  BEGIN EXP :{}".format(exp), timenow(),  0)
    
    expbar.set_description("EXP: {}".format(exp))
    A_CURRENT_EXP = exp
    
    e_res = run_experiments(e=exp)
    
    A_UPDATED_EXP = timenow()
    end_exp = time.time()
    update_log("  END EXP   :{}".format(exp), timenow(),  end_exp-start_exp)
    all_exps_results[exp] = e_res
    
    
play_done(s=final_sound)


Experiment::   0%|          | 0/7 [00:00<?, ?it/s]

Prep:   0%|          | 0/6 [00:00<?, ?it/s]

Vectzr::   0%|          | 0/5 [00:00<?, ?it/s]

Corpus::   0%|          | 0/10 [00:00<?, ?it/s]

Corpus::   0%|          | 0/10 [00:00<?, ?it/s]

...

Corpus::   0%|          | 0/10 [00:00<?, ?it/s]

Corpus::   0%|          | 0/10 [00:00<?, ?it/s]

Corpus::   0%|          | 0/10 [00:00<?, ?it/s]

Corpus::   0%|          | 0/10 [00:00<?, ?it/s]

23:10:09
